# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from numpy.linalg import matrix_power
from classes import Distribution, Operator, NpaOperator

# Bell Scenario Definition

In [2]:
N = 2

NA = N
NB = N
NX = N
NM = N

bell_scenario=(NA,NB,NX,NM)

A = np.array(range(NA))
X = np.array(range(NX))
B = np.array(range(NB))
M = np.array(range(NM))

# Distribution generator

for this particular one, we pick the optimal non-signaling distribution (not a quantum one) for the CHSH game.
$$
    p(a,b|x,m)= \frac{1}{2}\mathbb{1}_{a\oplus b = x\cdot m}
$$

In [3]:
vals = np.zeros(shape=(NA,NB,NX,NM))
for x in X:
    for m in M:
        for a in A:
            for b in B:
                lhs = (a+b)%2
                rhs = x*m
                val = 0.5*(lhs==rhs)
                vals[a,b,x,m] = val
p_perfect = Distribution(vals, bell_scenario)

## Local distributions

In [4]:
vals = np.zeros(shape=(NA,NB,NX,NM))
for x in X:
    for m in M:
        for a in A:
            for b in B:
                vals[a,b,x,m] = (a==x)*(b==m)
                vals[a,b,x,m] = 0.25
p_local = Distribution(vals, bell_scenario)

## Quantum distributions

In [5]:
omega = np.cos(np.pi/8)**2
vals = np.zeros(shape=(NA,NB,NX,NM))
for x in X:
    for m in M:
        for a in A:
            for b in B:
                if x*m:
                    vals[a,b,x,m] = 0.5*(a != b)*(2*omega - 1) + 0.5*(1 - omega)
                else:
                    vals[a,b,x,m] = 0.5*(a == b)*(2*omega - 1) + 0.5*(1 - omega)

p_quantum = Distribution(vals, bell_scenario)

For the first step of the NPA hierarchy, the $\Gamma$ matrix has the following form.
$$\Gamma\left(\mathcal{P}\right)=\left(\begin{array}{ccccccccc}
\overset{\left(\perp\right)}{\overbrace{1}} & \overset{\left(a=0,x=0\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=1,x=0\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=0,x=1\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(a=1,x=1\right)}{\overbrace{\Pr\left(a|x\right)}} & \overset{\left(b=0,m=0\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=1,m=0\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=0,m=1\right)}{\overbrace{\Pr\left(b|m\right)}} & \overset{\left(b=1,m=1\right)}{\overbrace{\Pr\left(b|m\right)}}\\
 & \Pr\left(a|x\right) & 0 & ? & ? & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  & \Pr\left(a|x\right) & ? & ? & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  & \Pr\left(a|x\right) & 0 & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  &  & \Pr\left(a|x\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right) & \Pr\left(a,b|x,m\right)\\
 &  &  &  &  & \Pr\left(b|m\right) & 0 & ? & ?\\
 &  &  &  &  &  & \Pr\left(b|m\right) & ? & ?\\
 &  &  &  &  &  &  & \Pr\left(b|m\right) & 0\\
 &  &  &  &  &  &  &  & \Pr\left(b|m\right)
\end{array}\right)$$

In [6]:
mu_vec = [cp.Variable() for _ in range(8)]
mu_gamma = cp.Variable()

In [7]:
p = (p_quantum * 1)
Γ = cp.bmat([[1, p.marginal(0,0,0), p.marginal(1,0,0), p.marginal(0,1,0), p.marginal(1,1,0), p.marginal(0,0,1), p.marginal(1,0,1), p.marginal(0,1,1), p.marginal(1,1,1)]
            ,[p.marginal(0,0,0), p.marginal(0,0,0), 0, mu_vec[0], mu_vec[1], p(0,0,0,0), p(0,1,0,0), p(0,0,0,1), p(0,1,0,1)]
            ,[p.marginal(1,0,0), 0, p.marginal(1,0,0), mu_vec[2], mu_vec[3], p(1,0,0,0), p(1,1,0,0), p(1,0,0,1), p(1,1,0,1)]
            ,[p.marginal(0,1,0), mu_vec[0], mu_vec[2], p.marginal(0,1,0), 0, p(0,0,1,0), p(0,1,1,0), p(0,0,1,1), p(0,1,1,1)]
            ,[p.marginal(1,1,0), mu_vec[1], mu_vec[3], 0, p.marginal(1,1,0), p(1,0,1,0), p(1,1,1,0), p(1,0,1,1), p(1,1,1,1)]
            ,[p.marginal(0,0,1), p(0,0,0,0), p(1,0,0,0), p(0,0,1,0), p(1,0,1,0), p.marginal(0,0,1), 0, mu_vec[4], mu_vec[5]]
            ,[p.marginal(1,0,1), p(0,1,0,0), p(1,1,0,0), p(0,1,1,0), p(1,1,1,0), 0, p.marginal(1,0,1), mu_vec[6], mu_vec[7]]
            ,[p.marginal(0,1,1), p(0,0,0,1), p(1,0,0,1), p(0,0,1,1), p(1,0,1,1), mu_vec[4], mu_vec[6], p.marginal(0,1,1), 0]
            ,[p.marginal(1,1,1), p(0,1,0,1), p(1,1,0,1), p(0,1,1,1), p(1,1,1,1), mu_vec[5], mu_vec[7], 0, p.marginal(1,1,1)]  ])

I = np.eye(9, 9)

In [8]:
constraints = list()
# constraints = [mu >= 0 for mu in mu_vec]
constraints += [Γ - mu_gamma * I >> 0]

In [9]:
objective = cp.Maximize(mu_gamma)
problem = cp.Problem(objective, constraints)
problem.solve()


3.709589349745165e-08

In [10]:
def npa_operator_generator(depth):
    npa_entries = list()
    def recursive_npa_entry(cur_depth, cur_entry, entry_list):
        if cur_depth == 0:
            return entry_list.append(cur_entry)
        else:
            for x in X:
                for a in A:
                    new_entry = cur_entry @ NpaOperator(Operator((a,x)), Operator.identity())
                    recursive_npa_entry(cur_depth - 1, new_entry, entry_list)
            for m in M:
                for b in B:
                    new_entry = cur_entry @ NpaOperator(Operator.identity(), Operator((b,m)))
                    recursive_npa_entry(cur_depth - 1, new_entry, entry_list)
    recursive_npa_entry(depth, NpaOperator(Operator.identity(), Operator.identity()), npa_entries)
    return npa_entries

def remove_duplicates(lst):
    new_lst = []
    for item in lst:
        if item not in new_lst:
            new_lst.append(item)
    return new_lst



In [11]:
npa_hierarchy_depth = 1
npa_operators = list()
for i in range(npa_hierarchy_depth + 1):
    npa_operators += remove_duplicates(npa_operator_generator(i))

In [12]:
existing_gamma_variables = dict()
for i in range(5):
    existing_gamma_variables[i] = f"{i}"

In [13]:
npa_operator_count = len(npa_operators)
gamma_matrix_entries = list()
for i in range(npa_operator_count):
    gamma_row = list()
    for j in range(npa_operator_count):
        gamma_row.append(npa_operators[i] @ npa_operators[j])
    gamma_matrix_entries.append(gamma_row)

In [19]:
class NpaHierarchy:
    def __init__(self, bell_scenario, npa_depth):
        if not isinstance(bell_scenario, tuple) or len(bell_scenario) != 4 or not all(isinstance(n, int) and n > 0 for n in bell_scenario):
            raise ValueError("bell_scenario must be a tuple of 4 positive integers")
        if not isinstance(npa_depth, int) or npa_depth <= 0:
            raise ValueError("npa_depth must be a positive integer")
        self.bell_scenario = bell_scenario
        NA,NB,NX,NM = self.bell_scenario
        self.npa_depth = npa_depth
        self.A = np.array(range(NA))
        self.B = np.array(range(NB))
        self.X = np.array(range(NX))
        self.M = np.array(range(NM))
        self.npa_operators = self.npa_operator_generator()
        self.gamma_operator_matrix = self.npa_gamma_matrix_generator()

    def npa_operator_generator(self):
        npa_entries = list()
        def recursive_npa_entry(cur_depth, cur_entry, entry_list):
            if cur_depth == 0:
                return entry_list.append(cur_entry)
            else:
                for x in self.X:
                    for a in self.A:
                        new_entry = cur_entry @ NpaOperator(Operator((a,x)), Operator.identity())
                        recursive_npa_entry(cur_depth - 1, new_entry, entry_list)
                for m in self.M:
                    for b in self.B:
                        new_entry = cur_entry @ NpaOperator(Operator.identity(), Operator((b,m)))
                        recursive_npa_entry(cur_depth - 1, new_entry, entry_list)
        for i in range(self.npa_depth + 1):
            recursive_npa_entry(i, NpaOperator(Operator.identity(), Operator.identity()), npa_entries)
        return NpaHierarchy._remove_duplicates(npa_entries) 
    
    @staticmethod
    def _remove_duplicates(lst):
        new_lst = []
        for item in lst:
            if item not in new_lst:
                new_lst.append(item)
        return new_lst
    
    def npa_gamma_matrix_generator(self):
        npa_operator_count = len(self.npa_operators)
        gamma_matrix_entries = list()
        for i in range(npa_operator_count):
            gamma_row = list()
            for j in range(npa_operator_count):
                gamma_row.append(self.npa_operators[i] @ self.npa_operators[j])
            gamma_matrix_entries.append(gamma_row)
        return gamma_matrix_entries
    
    def feasability(self, distribution):
        gamma_matrix_instance = list()
        variable_dict = dict()
        for i in range(len(self.gamma_operator_matrix)):
            gamma_matrix_instance_row = list()
            for j in range(len(self.gamma_operator_matrix[i])):
                current_operator = self.gamma_operator_matrix[i][j]
                if current_operator.is_variable():
                    if current_operator.__str__() in variable_dict:
                        current_value = variable_dict[current_operator.__str__()]
                    elif current_operator.conj().__str__() in variable_dict:
                        current_value = variable_dict[current_operator.conj().__str__()]
                    else:
                        current_value = cp.Variable()
                        variable_dict[current_operator.__str__()] = current_value
                else:
                    current_value = current_operator.evaluate(distribution)
                gamma_matrix_instance_row.append(current_value)
            gamma_matrix_instance.append(gamma_matrix_instance_row)
        return gamma_matrix_instance, variable_dict
        
        
        

In [23]:
npa = NpaHierarchy(bell_scenario, 1)

In [24]:
gamma, variable = npa.feasability(p_local)

In [26]:
gamma

[[1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0, Variable(()), Variable(()), 0.25, 0.25, 0.25, 0.25],
 [0.5, 0, 0.5, Variable(()), Variable(()), 0.25, 0.25, 0.25, 0.25],
 [0.5, Variable(()), Variable(()), 0.5, 0, 0.25, 0.25, 0.25, 0.25],
 [0.5, Variable(()), Variable(()), 0, 0.5, 0.25, 0.25, 0.25, 0.25],
 [0.5, 0.25, 0.25, 0.25, 0.25, 0.5, 0, Variable(()), Variable(())],
 [0.5, 0.25, 0.25, 0.25, 0.25, 0, 0.5, Variable(()), Variable(())],
 [0.5, 0.25, 0.25, 0.25, 0.25, Variable(()), Variable(()), 0.5, 0],
 [0.5, 0.25, 0.25, 0.25, 0.25, Variable(()), Variable(()), 0, 0.5]]